In [1]:
# blockparty_analysis package
import query_db, ops

In [2]:
import pandas as pd
import importlib
import pymongo
import yaml
import numpy as np

In [11]:
# to refresh changes to package
importlib.reload(query_db)

<module 'query_db' from '/Users/sarah/Documents/repos/block_party/analysis_tools/query_db.py'>

# Query Database

The scope of this notebook demonstrates a workflow that connects to the mongoDB and filters the collection of Block Party transcripts by a specific list of words using regex search.

The standard list of columns to export as a dataframe is defined by a YAML file. A API key is required to be able to reproduce the workflow, and should be saved outside of the repository. 

In [4]:
# Create an instance of the Constants class and pass the path to the YAML file
constants = ops.Constants('../analysis_tools/config.yaml')
# Access the data in the YAML file
columns_to_output = constants.constants['standard_column_list']


Filter database for any transcript that contains input word.

In [9]:
key_file_path = ops.pass_key('/Users/sarah/Documents/keys/mongo_key')

In [14]:
# Connect to database
client = pymongo.MongoClient(key_file_path)
filter = query_db.MongoDBFilter(client, 'community-board', 'transcripts_v4')

Structure of each dataframe will be a list of words to filter by, if a transcript contains any of the words it will pass through the filter.

In [7]:
zoning_word_list = ['[Zz]oning']

In [24]:
results = filter.filter_by_regex("properties.fullTranscript", zoning_word_list, projection=columns_to_output)

searching for: [Zz]oning


If the response collection is very large the conversion to a dataframe takes a very long time...

Todo: see if more efficient method to convert json to dataframe. (it took 34 minutes!)

In [14]:
df = query_db.flatten_docs_by_columns(results, columns_to_output)

Creating dataframe from database query...
Dataframe created...
Transformed col videoURL...
Transformed col normalizedName...
Transformed col publishDate...
Transformed col title...
Transformed col meetingType...
Transformed col fullTranscript...
Transformed col lengthSeconds...
Transformed col wordCountFullTranscript...
Transformed col wordCountSummary...


In [15]:
client.close()

Todo: turn the following into a helper function in addition to creating set from key sentences.

In [ ]:
# 5 older transcripts missing lengthSeconds 
df['lengthSeconds'].replace('NaN', np.NaN, inplace=True)
df['publishDate'] = pd.to_datetime(df['publishDate'], format='%Y-%m-%dT%H:%M:%S')
df['videoURL'] = df['videoURL'].apply(lambda x: f"https://www.youtube.com/watch?v={x}")

In [15]:
df.shape

(2052, 10)

In [28]:
df.to_pickle("../data/zoning_case_study.pkl")

In [ ]:
# todo: for each fulltranscript start to use text_search.py to unpack

In [18]:
df[df['normalizedName'] == 'Manhattan Community Board 8'].head()

,_id,videoURL,normalizedName,publishDate,title,meetingType,fullTranscript,lengthSeconds,wordCountFullTranscript,wordCountSummary
4,5fe7e243539a4d487dc18b6e,g1gcT-cVsWM,Manhattan Community Board 8,2020-12-10T00:00:00,Land Use Committee Meeting,"[[0.541478157043457, Zoning], [0.0916235074400...","Foreign me, okay, will tell me when it's a goo...",3:14:40,"{'building': 136, 'buildings': 100, 'flood': 9...","{'building': 47, 'flood': 38, 'buildings': 29,..."
6,5fe7e243539a4d487dc18b75,CL5991ZAYHY,Manhattan Community Board 8,2020-07-14T00:00:00,Landmarks Committee Meeting,"[[0.14057375490665436, Infrastructure], [0.099...",Let me know when you're ready for me to take d...,2:48:07,"{'building': 125, 'sidewalk': 37, 'feet': 36, ...","{'building': 44, 'equipment': 12, 'windows': 1..."
11,5fe7e244539a4d487dc18b88,iFmGCX6Sf_0,Manhattan Community Board 8,2020-11-20T00:00:00,Environment & Sanitation Meeting,"[[0.1319103091955185, Quality of Life], [0.123...","All right, let's just give it another minute h...",2:07:30,"{'rats': 50, 'monitoring': 36, 'rat': 34, 'pro...","{'rats': 13, 'monitoring': 9, 'sanitation': 7,..."
21,5fe7e244539a4d487dc18b98,Fl7m8RlNeg4,Manhattan Community Board 8,2020-11-25T00:00:00,Housing Committee Meeting,"[[0.7631369233131409, Housing], [0.10599086433...","And see me by the way will when we do, the min...",1:24:45,"{'housing': 128, 'rent': 38, 'state': 30, 'lan...","{'housing': 49, 'state': 8, 'land': 8, 'income..."
34,5fe7e245539a4d487dc18bb7,BcjOwgr4WEI,Manhattan Community Board 8,2020-12-01T00:00:00,Resource Sharing Session with MAS' Livable Nei...,"[[0.7714632153511047, Zoning], [0.137315973639...","Good, okay, so good evening, everyone and welc...",1:45:38,"{'zoning': 64, 'project': 64, 'building': 41, ...","{'project': 18, 'land': 15, 'zoning': 15, 'rev..."


In [2]:
df = pd.read_pickle("../data/zoning_case_study.pkl")

In [3]:
df.shape

(2052, 10)

In [20]:
df.drop(columns=["fullTranscript"]).to_csv("../data/zoning_case_study_metadata.csv", index=False)